In [16]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
import os

import json

import llama_cpp
import torch

from dotenv import load_dotenv
from openai import OpenAI

from curverag import utils
from curverag.curverag import CurveRAG, DEFAULT_ENTITY_TYPES, DEFAULT_GLINER_MODEL, DEFAULT_SENTENCE_TRANSFORMER_MODEL
from curverag.graph import KnowledgeGraph
from curverag.atth.kg_dataset import KGDataset
from curverag.atth.models.hyperbolic import AttH
from curverag.eval import evaluation, queries, context, expected_output

load_dotenv() 

True

# Explore dataset

In [18]:
with open('/Users/nathan/Documents/projects/datasets/2WikiMultihopQA/train.json', 'rb') as f:
    train = json.load(f)

with open('/Users/nathan/Documents/projects/datasets/2WikiMultihopQA/dev.json', 'rb') as f:
    dev = json.load(f)

with open('/Users/nathan/Documents/projects/datasets/2WikiMultihopQA/test.json', 'rb') as f:
    test = json.load(f)

In [19]:
train[0].keys()

dict_keys(['_id', 'type', 'question', 'context', 'supporting_facts', 'evidences', 'answer'])

In [20]:
train[0]['question']

'Are director of film Move (1970 Film) and director of film Méditerranée (1963 Film) from the same country?'

In [21]:
train[0]['answer']

'no'

In [22]:
train[0]['context']

[['Stuart Rosenberg',
  ['Stuart Rosenberg (August 11, 1927 – March 15, 2007) was an American film and television director whose motion pictures include "Cool Hand Luke" (1967), "Voyage of the Damned" (1976), "The Amityville Horror" (1979), and "The Pope of Greenwich Village" (1984).',
   'He was noted for his work with actor Paul Newman.']],
 ['Méditerranée (1963 film)',
  ['Méditerranée is a 1963 French experimental film directed by Jean-Daniel Pollet with assistance from Volker Schlöndorff.',
   'It was written by Philippe Sollers and produced by Barbet Schroeder, with music by Antione Duhamel.',
   'The 45 minute film is cited as one of Pollet\'s most influential films, which according to Jonathan Rosenbaum directly influenced Jean-Luc Goddard\'s "Contempt", released later the same year.',
   'Footage for the film was shot around the Mediterranean, including at a Greek temple, a Sicilian garden, the sea, and also features a fisherman, a bullfighter, and a girl on an operating table

# Run eval

In [23]:
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

In [24]:
entity_types = ['people', 'locations', 'entities', 'movies', 'directors']
rag = CurveRAG(
    openai_client=client,
    entity_types=entity_types,
)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

In [10]:
new_docs = []
for d in train[0]['context']:
    d_ = str(d).replace('[', '').replace(']', '. ').replace('\'', '')
    new_docs.append(d_)

In [25]:
new_docs = []
for d in train:
    d_ = str(d['context']).replace('[', '').replace(']', '. ').replace('\'', '')
    new_docs.append(d_)

In [26]:
rag.fit(new_docs[:100], dataset_name='movies')

creating graph


  1%|▍                                                                   | 1/138 [00:12<28:55, 12.67s/it]

upserting


  1%|▉                                                                   | 2/138 [00:21<23:42, 10.46s/it]

upserting


...

100%|██████████████████████████████████████████████████████████████████| 138/138 [27:21<00:00, 11.89s/it]
2025-06-22 12:43:53,856 INFO     Saving logs in: ./logs/06_22/movies/AttH_12_43_53
2025-06-22 12:43:53,857 INFO     	 (799, 484, 799)
2025-06-22 12:43:53,927 INFO     Total number of parameters 3221082
2025-06-22 12:43:53,931 INFO     	 Start training


upserting
num nodes 799
unique node ids {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214

train loss: 100%|██████████████████████████████████████| 1252/1252 [00:10<00:00, 124.46ex/s, loss=2.0961]
2025-06-22 12:44:03,994 INFO     	 Epoch 0 | average train loss: 1.3923
2025-06-22 12:44:04,095 INFO     	 Epoch 0 | average valid loss: 1.7016
train loss: 100%|██████████████████████████████████████| 1252/1252 [00:07<00:00, 160.66ex/s, loss=1.6575]
2025-06-22 12:44:11,894 INFO     	 Epoch 1 | average train loss: 1.7686
2025-06-22 12:44:11,990 INFO     	 Epoch 1 | average valid loss: 0.9531
train loss: 100%|██████████████████████████████████████| 1252/1252 [00:06<00:00, 197.78ex/s, loss=0.8719]
2025-06-22 12:44:18,322 INFO     	 Epoch 2 | average train loss: 1.0416
2025-06-22 12:44:18,413 INFO     	 Epoch 2 | average valid loss: 0.4587
2025-06-22 12:44:18,431 INFO     	 valid MR: 201.85 | MRR: 0.013 | H@1: 0.000 | H@3: 0.000 | H@10: 0.013
2025-06-22 12:44:18,432 INFO     	 Epoch 2 | average valid mmr: 0.0126
2025-06-22 12:44:18,432 INFO     	 Saving model at epoch 2 in ./logs/06_22

returning model


Batches:   0%|          | 0/25 [00:00<?, ?it/s]

In [27]:
rag.graph.nodes

[Node(id=1, name='Stuart Rosenberg', description='American film and television director American film and television director', alias=['Stuart Rosenberg (August 11, 1927 – March 15, 2007)', 'Stuart Rosenberg'], additional_information=["Directed 'Cool Hand Luke', 'Voyage of the Damned', 'The Amityville Horror', 'The Pope of Greenwich Village'"]),
 Node(id=2, name='Méditerranée', description='1963 French experimental film', alias=['Méditerranée (1963 film)'], additional_information=['Directed by Jean-Daniel Pollet', 'Written by Philippe Sollers', 'Produced by Barbet Schroeder']),
 Node(id=3, name='Move', description='1970 American comedy film', alias=['Move (1970 film)'], additional_information=['Starring Elliott Gould, Paula Prentiss, and Geneviève Waïte', 'Directed by Stuart Rosenberg']),
 Node(id=4, name='Ian Barry', description='Australian director of film and TV Australian director of film and TV Australian director of film and TV Australian director of film and TV Australian direct

In [28]:
rag.graph.edges

[Edge(source=1, target=3, name='directed', is_directed=True, description='Directed the film', notes=["Directed 'Move' (1970)"]),
 Edge(source=1, target=2, name='directed', is_directed=True, description='Directed the film', notes=["Directed 'Méditerranée' with assistance"]),
 Edge(source=2, target=9, name='influenced', is_directed=False, description='Influenced another film', notes=["Influenced 'Contempt' by Jean-Luc Godard"]),
 Edge(source=9, target=4, name='associated', is_directed=False, description='Approaches to filmmaking', notes=['Burlesque and melancholic elements', 'Based on texts by French poet Francis Ponge']),
 Edge(source=11, target=13, name='directed', is_directed=True, description='Directed the film', notes=["Directed 'The Falcon Takes Over'"]),
 Edge(source=12, target=14, name='directed', is_directed=True, description='Directed the film', notes=["Directed 'The Falcon Strikes Back'"]),
 Edge(source=13, target=11, name='starred', is_directed=False, description='Starred in 

In [29]:
print('Question:', train[0]['question'])
resp = rag.query(train[0]['question'], traversal = 'pp')
resp

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Question: Are director of film Move (1970 Film) and director of film Méditerranée (1963 Film) from the same country?
entities ['Move', 'Méditerranée']
Are AUX VBP
director NOUN NN
of ADP IN
film NOUN NN
Move PROPN NNP
( PUNCT -LRB-
1970 NUM CD
Film PROPN NNP
) PUNCT -RRB-
and CCONJ CC
director NOUN NN
of ADP IN
film NOUN NN
Méditerranée PROPN NNP
( PUNCT -LRB-
1963 NUM CD
Film PROPN NNP
) PUNCT -RRB-
from ADP IN
the DET DT
same ADJ JJ
country NOUN NN
? PUNCT .
pos_tags ['director', 'film', 'Move', 'Film', 'director', 'film', 'Méditerranée', 'Film', 'country']


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

graph nodes retrieved ['Méditerranée', 'Move', 'At the Movies', 'American Film Institute', 'The Actors', 'The Da Vinci Code (film)', 'A Far Country', 'Committed (film)', 'Blowback (film)', 'Clouds (film)']
entites for edges ['Move', 'Méditerranée', 'director', 'film', 'Move', 'Film', 'director', 'film', 'Méditerranée', 'Film', 'country']


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/25 [00:00<?, ?it/s]

similar_edges ['influenced', 'associated', 'directed', 'starred', 'reprises role', 'produced']
similar_node_ids [140, 330, 339, 401, 2, 764, 765, 3, 131, 336]
similar_node_ids graph nodes retrieved ['Méditerranée', 'Move', 'At the Movies', 'Neil Young', 'Judy Garland', 'American Film Institute', 'A Pretty Girl Milking Her Cow', 'The Actors', 'Committed (film)', 'Blowback (film)']
***************************************!!******************************
***************************************!!******************************
***************************************!!******************************
***************************************!!******************************
***************************************!!******************************
***************************************!!******************************
***************************************!!******************************
***************************************!!******************************
***************************************!!

2025-06-22 12:50:16,841 INFO     HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"


'The director of the film *Méditerranée* (1963) is Jean-Daniel Pollet, and the director of the film *Move* (1970) is Stuart Rosenberg. *Méditerranée* is a French film, indicating that Jean-Daniel Pollet is from France. In contrast, *Move* is an American film, suggesting that Stuart Rosenberg is from the United States. Therefore, the directors of these two films are not from the same country.'

In [30]:
print('Question:', train[0]['question'])
resp = rag.query(train[0]['question'])
resp

Question: Are director of film Move (1970 Film) and director of film Méditerranée (1963 Film) from the same country?
entities ['Move', 'Méditerranée']
Are AUX VBP
director NOUN NN
of ADP IN
film NOUN NN
Move PROPN NNP
( PUNCT -LRB-
1970 NUM CD
Film PROPN NNP
) PUNCT -RRB-
and CCONJ CC
director NOUN NN
of ADP IN
film NOUN NN
Méditerranée PROPN NNP
( PUNCT -LRB-
1963 NUM CD
Film PROPN NNP
) PUNCT -RRB-
from ADP IN
the DET DT
same ADJ JJ
country NOUN NN
? PUNCT .
pos_tags ['director', 'film', 'Move', 'Film', 'director', 'film', 'Méditerranée', 'Film', 'country']


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

graph nodes retrieved ['Méditerranée', 'Move', 'At the Movies', 'American Film Institute', 'The Actors', 'The Da Vinci Code (film)', 'A Far Country', 'Committed (film)', 'Blowback (film)', 'Clouds (film)']
scores tensor([1.2203, 1.3070, 1.3187, 1.3485, 1.4398, 1.4547, 1.4754, 1.5413, 1.5517,
        1.5782], dtype=torch.float64)
scores tensor([2.1295, 2.2235, 2.2241, 2.2264, 2.3177, 2.3220, 2.4058, 2.4134, 2.4222,
        2.4255], dtype=torch.float64)
scores tensor([0.9095, 0.9110, 0.9120, 0.9147, 0.9166, 0.9187, 0.9194, 0.9206, 0.9215,
        0.9226], dtype=torch.float64)
scores tensor([1.0979, 1.2987, 1.3497, 1.3790, 1.4293, 1.4593, 1.4777, 1.5189, 1.5224,
        1.5495], dtype=torch.float64)
scores tensor([1.0992, 1.9681, 1.9968, 2.0183, 2.0561, 2.0810, 2.1175, 2.1190, 2.1274,
        2.1335], dtype=torch.float64)
scores tensor([2.2994e-07, 2.7842e+00, 2.8291e+00, 2.8668e+00, 2.9589e+00, 2.9795e+00,
        2.9971e+00, 3.0077e+00, 3.0098e+00, 3.0137e+00], dtype=torch.float64)
scor

2025-06-22 12:57:27,478 INFO     HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"


'The director of the film *Move* (1970) is Stuart Rosenberg, who is American, while the director of *Méditerranée* (1963) is Jean-Daniel Pollet, who is French. Therefore, they are not from the same country, as one is from the United States and the other from France. [Sources: 1, 2, 3]'

In [24]:
print('Question:', queries[1])
resp = rag.query(queries[1])
resp

Question: What gender is Jill?Give me the character arc of Sam
entities ['Jill', 'Sam']


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[[np.int64(1), np.int64(2)]]
node_ids [tensor([9, 7]), tensor([4, 0, 5])]
node_ids [9, 7, 4, 0, 5]
***************************************!!******************************
***************************************!!******************************
***************************************!!******************************
***************************************!!******************************
***************************************!!******************************
***************************************!!******************************

You are a helpful assistant analyzing the given input data to provide an helpful response to the user query.

# USER QUERY
What gender is Jill?Give me the character arc of Sam

# Context:
KnowledgeGraph Overview
  There are 6 entities and 7 relationships in this graph.

Entities in the graph:
  • 'Jill':
      The entity has the following description: Jill is an adult sibling of Jack.
      It can also be referred to as: Jill
      It has the following additional 

Llama.generate: 1005 prefix-match hit, remaining 1 prompt tokens to eval
llama_perf_context_print:        load time =    9163.97 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =    9254.43 ms /   100 runs   (   92.54 ms per token,    10.81 tokens per second)
llama_perf_context_print:       total time =    9668.53 ms /   101 tokens


'Jill is an adult sibling of Jack. As for the character arc of Sam, Sam is a friend of Jack and Jill, and they helped Sam find love. Sam then moved to Romania with their friend [1]. Sam found love in Romania, marking a significant change in their life [2].\n\nReferences:\n[1] Entity: Romania\n[2] Relationship: Sam moved to Romania with their friend\n\nNote: The answer is based on the information provided in the input data, and it is'

In [25]:
print('Question:', queries[2])
resp = rag.query(queries[2])
resp

Question: What location is the story set in
entities []


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[[]]
node_ids []
node_ids []

You are a helpful assistant analyzing the given input data to provide an helpful response to the user query.

# USER QUERY
What location is the story set in

# Context:
KnowledgeGraph Overview
  There are 0 entities and 0 relationships in this graph.

Entities in the graph:

Relationships between nodes:



# INSTRUCTIONS
Your goal is to provide a response to the user query using the relevant information in the input data:
- the "Entities" and "Relationships" tables contain high-level information. Use these tables to identify the most important entities and relationships to respond to the query.
- the "Sources" list contains raw text sources to help answer the query. It may contain noisy data, so pay attention when analyzing it.

Follow these steps:
1. Read and understand the user query.
2. Look at the "Entities" and "Relationships" tables to get a general sense of the data and understand which information is the most relevant to answer the query.
3. Carefu

Llama.generate: 28 prefix-match hit, remaining 333 prompt tokens to eval
llama_perf_context_print:        load time =    9163.97 ms
llama_perf_context_print: prompt eval time =    3022.72 ms /   333 tokens (    9.08 ms per token,   110.17 tokens per second)
llama_perf_context_print:        eval time =    4559.04 ms /    99 runs   (   46.05 ms per token,    21.72 tokens per second)
llama_perf_context_print:       total time =    7756.66 ms /   432 tokens


"Based on the given input data, no location is mentioned in the data. Therefore, it is not possible to determine the location of the story based on the provided information. [1]\n\nSource: [1] - There is no source ID provided in the input data. \n\nNote: I've followed the instructions and tried my best to provide a helpful response to the user query. Please let me know if I can improve anything. \n\nPlease let me know if the response is accurate and if there's"

In [119]:
print('Question:', queries[3])
resp = rag.query(queries[3])
resp

Question: What location is the story set in
entities []


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

graph nodes retrieved []
Found no embedding indx for entities, doing non KG-RAG result


'N/A'

In [29]:
print('Question:', queries[4])
resp = rag.query(queries[4])
resp

Question: Tell me how old Jack is, what the gender of Jill is, and then give me the character arcs of Sam and where the stoty is set
entities ['Jack', 'Jill', 'Sam']


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[[np.int64(0), np.int64(1), np.int64(2)]]
node_ids [tensor([4, 2]), tensor([9, 7]), tensor([4, 0, 5])]
node_ids [4, 2, 9, 7, 4, 0, 5]
***************************************!!******************************
***************************************!!******************************
***************************************!!******************************
***************************************!!******************************
***************************************!!******************************
***************************************!!******************************
***************************************!!******************************

You are a helpful assistant analyzing the given input data to provide an helpful response to the user query.

# USER QUERY
Tell me how old Jack is, what the gender of Jill is, and then give me the character arcs of Sam and where the stoty is set

# Context:
KnowledgeGraph Overview
  There are 7 entities and 10 relationships in this graph.

Entities in the gra

Llama.generate: 57 prefix-match hit, remaining 1147 prompt tokens to eval
llama_perf_context_print:        load time =    9163.97 ms
llama_perf_context_print: prompt eval time =   10635.08 ms /  1147 tokens (    9.27 ms per token,   107.85 tokens per second)
llama_perf_context_print:        eval time =    4664.64 ms /    99 runs   (   47.12 ms per token,    21.22 tokens per second)
llama_perf_context_print:       total time =   15811.75 ms /  1246 tokens


"Based on the given data, we can answer the user query as follows:\nJack is 45 years old [1]. The gender of Jill is not explicitly mentioned in the given data, so we cannot determine it. As for Sam's character arc, we do not have enough information to provide a comprehensive analysis. The story is set in Romania [2], where Sam and their friend moved to. \n\nReferences:\n[1] - Entity description of Jack\n[2] - Relationship between Sam and Romania"

# networkx

In [120]:
import networkx as nx
import numpy as np
import scipy as sp

In [174]:
np.random.randint(10)

5

In [175]:
G = nx.Graph()
G.add_nodes_from([n.id for n in rag.graph.nodes])
edges = [(e.source, e.target, {'e_type':e.name, 'weight':np.random.randint(10)}) for e in rag.graph.edges]
G.add_edges_from(edges)
weight= 'weight'

In [176]:
G.edges(data=True)

EdgeDataView([(1, 10, {'e_type': 'directed', 'weight': 9}), (2, 3, {'e_type': 'directed', 'weight': 8}), (2, 4, {'e_type': 'assisted', 'weight': 4}), (2, 5, {'e_type': 'written by', 'weight': 6}), (2, 6, {'e_type': 'produced by', 'weight': 0}), (2, 7, {'e_type': 'music by', 'weight': 7}), (2, 8, {'e_type': 'influenced', 'weight': 1}), (8, 9, {'e_type': 'influenced', 'weight': 3}), (10, 11, {'e_type': 'starring', 'weight': 6}), (10, 12, {'e_type': 'starring', 'weight': 5}), (10, 13, {'e_type': 'starring', 'weight': 3}), (10, 14, {'e_type': 'written by', 'weight': 7}), (10, 15, {'e_type': 'written by', 'weight': 6}), (14, 14, {'e_type': 'adapted from', 'weight': 5})])

In [162]:
G.number_of_nodes()

21

In [163]:
N = len(G)
N

21

In [177]:
nodelist = list(G)
nodelist

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]

In [178]:
A = nx.to_scipy_sparse_array(G, nodelist=nodelist, weight=weight, dtype=float)
A

<Compressed Sparse Row sparse array of dtype 'float64'
	with 27 stored elements and shape (21, 21)>

In [179]:
A.toarray()

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 9., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 0., 8., 4., 6., 0., 7., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 8., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 4., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 6., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 7., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 3., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 3., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [9., 0., 0., 0., 0., 0., 0., 0., 0., 0., 6., 5., 3., 7., 6., 0.,
        0., 0., 0.,

In [166]:
A.toarray()

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 0., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 0.,
        0., 0., 0.,

In [180]:
S = A.sum(axis=1)
S

array([ 9., 26.,  8.,  4.,  6.,  0.,  7.,  4.,  3., 36.,  6.,  5.,  3.,
       12.,  6.,  0.,  0.,  0.,  0.,  0.,  0.])

In [181]:
1/S[S != 0]

array([0.11111111, 0.03846154, 0.125     , 0.25      , 0.16666667,
       0.14285714, 0.25      , 0.33333333, 0.02777778, 0.16666667,
       0.2       , 0.33333333, 0.08333333, 0.16666667])

In [182]:
S[S != 0] = 1.0 / S[S != 0]
S

array([0.11111111, 0.03846154, 0.125     , 0.25      , 0.16666667,
       0.        , 0.14285714, 0.25      , 0.33333333, 0.02777778,
       0.16666667, 0.2       , 0.33333333, 0.08333333, 0.16666667,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        ])

In [183]:
Q = sp.sparse.csr_array(sp.sparse.spdiags(S.T, 0, *A.shape))
Q

<Compressed Sparse Row sparse array of dtype 'float64'
	with 14 stored elements and shape (21, 21)>

In [184]:
Q.toarray()

array([[0.11111111, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.03846154, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.125     , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , 0.25      , 0.        ,
        0.        , 0.        , 0.        , 0.       

In [185]:
A = Q @ A
A

<Compressed Sparse Row sparse array of dtype 'float64'
	with 25 stored elements and shape (21, 21)>

In [186]:
A = Q @ A


In [187]:
A.toarray()

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.11111111,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.01183432, 0.00591716, 0.00887574,
        0.        , 0.01035503, 0.00147929, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.125     , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.25      , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.       

In [188]:
x = np.repeat(1.0 / N, N)
x

array([0.04761905, 0.04761905, 0.04761905, 0.04761905, 0.04761905,
       0.04761905, 0.04761905, 0.04761905, 0.04761905, 0.04761905,
       0.04761905, 0.04761905, 0.04761905, 0.04761905, 0.04761905,
       0.04761905, 0.04761905, 0.04761905, 0.04761905, 0.04761905,
       0.04761905])

In [189]:
p = np.repeat(1.0 / N, N)
p

array([0.04761905, 0.04761905, 0.04761905, 0.04761905, 0.04761905,
       0.04761905, 0.04761905, 0.04761905, 0.04761905, 0.04761905,
       0.04761905, 0.04761905, 0.04761905, 0.04761905, 0.04761905,
       0.04761905, 0.04761905, 0.04761905, 0.04761905, 0.04761905,
       0.04761905])

In [190]:
dangling_weights = p

In [151]:
is_dangling = np.where(S == 0)[0]
is_dangling

array([15, 16, 17, 18, 19, 20])

In [152]:
0.5*0.1

0.05

In [153]:
alpha = 0.85

In [154]:
x

array([0.04761905, 0.04761905, 0.04761905, 0.04761905, 0.04761905,
       0.04761905, 0.04761905, 0.04761905, 0.04761905, 0.04761905,
       0.04761905, 0.04761905, 0.04761905, 0.04761905, 0.04761905,
       0.04761905, 0.04761905, 0.04761905, 0.04761905, 0.04761905,
       0.04761905])

In [160]:
A.toarray()#.sum(axis=1)

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 1.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.02777778, 0.02777778, 0.02777778,
        0.02777778, 0.02777778, 0.02777778, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.       

In [118]:
0.04761905*1

0.04761905

In [116]:
x @ A

array([0.00793651, 0.26190476, 0.00793651, 0.00793651, 0.00793651,
       0.00793651, 0.00793651, 0.05555556, 0.02380952, 0.26190476,
       0.00793651, 0.00793651, 0.00793651, 0.03174603, 0.00793651,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        ])

In [104]:
x_new = alpha * (x @ A + sum(x[is_dangling]) * dangling_weights) + (1 - alpha) * p


In [108]:
x_new

array([0.02545351, 0.24132653, 0.02545351, 0.02545351, 0.02545351,
       0.02545351, 0.02545351, 0.06592971, 0.03894558, 0.24132653,
       0.02545351, 0.02545351, 0.02545351, 0.04569161, 0.02545351,
       0.01870748, 0.01870748, 0.01870748, 0.01870748, 0.01870748,
       0.01870748])

In [ ]:
edge_types